In [1]:
from aide_design.play import*
from aide_design import cdc_functions as cdc
from aide_design.unit_process_design.prefab import lfom_prefab_functional as lfom
#Below are the items that were imported by the code above so that you know what abbreviations to use in your code.

# Third-party imports
#import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib

# AIDE imports
#import aide_design
#import aide_design.pipedatabase as pipe
#from aide_design.units import unit_registry as u
#from aide_design import physchem as pc
#import aide_design.expert_inputs as exp
#import aide_design.materials_database as mat
#import aide_design.utility as ut
#import aide_design.k_value_of_reductions_utility as k
#import aide_design.pipeline_utility as pipeline
#import warnings


In [2]:
## Set parameters of plant below

In [3]:
Q_plant=1*u.L/u.s
T = u.Quantity(20,u.degC)

Headloss=10*u.cm
C_stock_PACl=70*u.gram/u.L
NuPaCl = cdc.viscosity_kinematic_pacl(C_stock_PACl,T)
DesiredCl_Concentration=3*u.mg/u.L
RatioError = 0.1
KMinor = 2

In [4]:
Q_CDC=(Q_plant*DesiredCl_Concentration/C_stock_PACl).to(u.mL/u.s)
print(Q_CDC) ##mL/sec
print(Q_CDC.to(u.L/u.hour)) ##L/hr

0.04286 milliliter / second
0.1543 liter / hour


In [5]:
##Find diameter of dooosing toob

In [6]:
DiamTubeArray = np.array(np.arange(1/16,3/16,1/16)) * u.inch ## 1/16 or 1/8 inch tubing


VelTubeMax = (((RatioError * 2 * Headloss * pc.gravity) / KMinor)**(1/2)).to(u.meter/u.s)
print('The maximum average velocity in a dosing tube is',VelTubeMax)

The maximum average velocity in a dosing tube is 0.3132 meter / second


In [7]:
##Flow rates for tubes

In [8]:
def flow_cdc_max(diam, VelTubeMax): 
    Flow = pc.area_circle(diam) * (VelTubeMax)
    return Flow
        
FlowMaxArray = flow_cdc_max(DiamTubeArray, VelTubeMax).to(u.mL/u.s)
FlowMaxArray

<Quantity([ 0.61983727  2.47934908], 'milliliter / second')>

## Lengths for each tube
Here you need to use the actual maximum flow rate based on the chemical dose that you want to apply. You mistakenly used the maximum flow that maintains linearity. 

In [9]:
def length_tube(flow_max, diam, headloss_max, NuPaCl, k_minor):
    "Returns the length of tube necessary to handle the maximum flow."
    L = (((pc.gravity * headloss_max * np.pi * diam**4)/
                           (128 * NuPaCl * flow_max))-
                    ((k_minor * flow_max)/
                             (16 * np.pi * NuPaCl)))
    return L 
length_dosingtube=length_tube(FlowMaxArray, DiamTubeArray,Headloss, NuPaCl,KMinor)
print(length_dosingtube.to(u.m).magnitude)

[ 0.20591553  0.82366212]


In [10]:
##Both are reasonably lengthed. .22 would be easier to wrap around so I conclude that we should use the 1/16th diameter tubing. 
